# Metaklasy

Wszystko jest obiektem - tak jak instancje mają swoje klasy/typy które określają ich zachowanie, tak klasy mają swoje klasy/typy, które określają ich zachowanie. Są to właśnie metaklasy. Podstawiową metaklasą jest type (tak, to type, które mówi nam jakiego typu są obiekty); type jest metaklasą dla samego siebie.

Metaklasy to funkcje albo klasy, które powinny zwrócić nową klasę 

In [11]:
print type(2)
print type(int)
print type(type)

Kotek = type('Kotek', (), {'co_lubi': 'mleko', 'f': lambda self: self.co_lubi})
kotek  = Kotek()
kotek.co_lubi
isinstance(kotek, int)
Kotek.mro()
kotek.f()


<type 'int'>
<type 'type'>
<type 'type'>


'mleko'

In [16]:
# metaklasa przyjmuje 3 argumenty -
# nazwę klasy, listę klas bazowych i namespace - atrybuty i metody
def metaklasa(name, bs, ns):
    print 'metaklasa pozdrawia'
    print 'name: ', name
    print 'bs', bs
    print 'ns', ns
    return type(name, bs, ns)


# metaklasę podpina się w pythonie 2 do atrybutu __metaclass__
class A(object):
    __metaclass__ = metaklasa

    
print 30 * '='
class B(A):
    __metaclass__ = metaklasa
    x = 23
    def f(self):
        print self

metaklasa pozdrawia
name:  A
bs (<type 'object'>,)
ns {'__module__': '__main__', '__metaclass__': <function metaklasa at 0x7f5bbc6f8b18>}


In [ ]:
# generalnie metaklasa wcale nie musi wywoływać type i tworzyć obiektu określonej klasy,
# może na przykałd zwracać int

def metaklasa(*a):
    return 2

class A:
    __metaclass__ = metaklasa
    

print A

In [26]:
# w praktyce, metaklalsy to klasy, które dziedziczą po type
# (oczywiscie nie muszą, ale wtedy trzeba samemu zatroszyc się o odpowiednie tworzenie
# obiektów)

class Meta(type):
    def __new__(cls, name, bs, ns):
        # new to classmethod  - cls jest metaklasą Meta
        # przechwytywanie procesu tworzenia klasy - tego co będzie przypisane do zmiennej
        # o znawie name
        print 'tworzenie obiektu'
        # można wpływac na atrybuty, mro, i nazwę klasy, ale teraz nie zrobimy nic
        return super(Meta, cls).__new__(cls, name, bs, ns)
    
    def __init__(self, name , bs, ns):
        # na tym etapie self to juz utworzona klasa
        # inicjalizacja klasy - nazwa jest już nadana, to za późno, żeby to zmienić,
        # tak samo klasy bazowe i atrybuty
        print "inicjalizacja"
        print name, bs, ns
        name = name.upper()
        bs = ()
        ns = {'x': 24}
        return super(Meta, self).__init__(name, bs, ns)
    
    def __call__(self, *a, **kw):
        # tak jak a() wywoływało metodę __call__ z klasy na rzecz tego obiektu,
        # tak A() wywołuje __call__ metaklasy, na rzecz tej klasy
        # jeżeli klasa nie jest wywoływalna (nie ma __call__ w metaklasie to nie można
        # stworzyć instancji klasy)
        print 'Meta.__call__'
        return super(Meta, self).__call__(*a, **kw)
        

class A(object):
    __metaclass__ = Meta
    
A()
    

class Aasdf(A):  # jeżeli klasa bazowa nie ma zdefiniowanej metaklasy,
    x = 23       # to uzywana jest ta z klasy bazowej - metaklasy są dziedziczone
    
print Aasdf
print Aasdf.__name__  # __init__ nie wpływa na nazwę klasy
print Aasdf.mro()  # __init__ nie wpływa na mro
print dir(Aasdf)  # __init__ nie wpływa na listę atrybutów
print Aasdf.x  # __init__ nie wpływa na wartosći atrybutów

a = Aasdf()


print '=============='
# alternatywnie można towrzyć klasy wywołujac metaklasę - dokładnie tak jak tworząc instancję klasy wywoływać klase -
# bo w rzeczywistości klasy są instancjami metaklas
Klaska = Meta('Klaska', (), {})
print Klaska

tworzenie obiektu
inicjalizacja
A (<type 'object'>,) {'__module__': '__main__', '__metaclass__': <class '__main__.Meta'>}
Meta.__call__
tworzenie obiektu
inicjalizacja
Aasdf (<class '__main__.A'>,) {'x': 23, '__module__': '__main__'}
<class '__main__.Aasdf'>
Aasdf
[<class '__main__.Aasdf'>, <class '__main__.A'>, <type 'object'>]
['__class__', '__delattr__', '__dict__', '__doc__', '__format__', '__getattribute__', '__hash__', '__init__', '__metaclass__', '__module__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'x']
23
Meta.__call__
tworzenie obiektu
inicjalizacja
Klaska () {}
<class '__main__.Klaska'>


In [ ]:
In [46]: class BSCMeta(type):
    ...:     def __getitem__(self, item):
    ...:         print self
    ...:         if isinstance(item, slice):
    ...:             return [self(i) for i in range(item.start, item.stop, item.step or 1)]
    ...:         return self(item)
    
    
    
In [49]: class BSC:
    ...:     __metaclass__ = BSCMeta
    ...:     def __init__(self, id):
    ...:         self.id = id
    ...:         


## klasy abstrakcyjne
moduł abc (Abstract Base Classes) zawiera metaklasę ABCMeta, która jest uzywana do tworzenia 
m.in metod abstrakcyjnych, ale równiez kilku innyc hciekawych rzeczy

In [22]:
# można dodac 'virtualne klasy bazowe' przy pomocy funkcji register

import abc


class Sekwencja:
    __metaclass__ = abc.ABCMeta
    
    
Sekwencja.register(tuple)  # krotka będzie widziana jako podklasa Sekwencji


print tuple.mro()  # Sekwencji nie ma w mro


print issubclass(tuple, Sekwencja)  # ale i tak pokazuje że krotka jest podklasą
print isinstance((), Sekwencja)

print issubclass(list, Sekwencja)  # lista nie jest tak postrzegana
print isinstance([], Sekwencja)

print 40 * '='

class Sekwencja2(Sekwencja):
    pass

print issubclass(tuple, Sekwencja2)  # funkcja register nie działa dla klas dizedziczącyk po
print isinstance((), Sekwencja2)     # wirtualnej klasie bazowej

[<type 'tuple'>, <type 'object'>]
True
True
False
False
False
False


In [23]:
# alternatywnie można zaimplementować metodę __subclasshook__ jako classmethod

import abc


class Sekwencja :
    __metaclass__ = abc.ABCMeta
    
    @classmethod
    def __subclasshook__(cls, subclass):
        if subclass in [tuple, list]:  # krotki i listy są uważane za podklasy
            return True
        return False


print tuple.mro()  # Sekwencji nie ma w mro


print issubclass(tuple, Sekwencja)  # ale i tak pokazuje że krotka jest podklasą
print isinstance((), Sekwencja)


print 40 * '='

class Sekwencja2(Sekwencja):
    pass

print issubclass(tuple, Sekwencja2)  # działa dla klasy pochodnej - register nie działało
print isinstance((), Sekwencja2)


# metodę __subclasshook__ i funkcję register można łączyć - wtedy "lista podklas"
# to suma zbiorów klas zarejestrowanych i tych określonych przez metodę __subclasshook__

[<type 'tuple'>, <type 'object'>]
True
True
True
True


### metody i property abstrakcyjne

In [19]:
# tak drzewiej bywało:

class Abstrakcyjna(object):
    # dużo metod
    def metoda_abstrakcyjna(self):
        raise NotImplementedError() # generalnie zadziała
        
        
class NieAbstrakcyjna(Abstrakcyjna):
    # programista naspisuje inne metody - tylko zapomniał o tej o nazwie metoda_abstrakcyjna
    pass
        
    
a = NieAbstrakcyjna()  # przejdzie, a nie powinno, bo programista nie nadpisał metody abstrakcyjnej

# w zupełnie innym pliku 2000 linii kodu dalej
a.metoda_abstrakcyjna()  # wykrzaczy się - dużo debugowania
        


NotImplementedError: 

In [21]:
# metody abstrakcyjne muszą być nadpisane w klasach bazowych

import abc

class A:
    __metaclass__ = abc.ABCMeta  # do klas abstrakcyjnych musi być dodana ABCMeta
    
    @abc.abstractmethod
    def f(self):
        pass
    

# a = A()  # error

class B(A):
    pass

# b = B()  # error


class C(B):
    def f(self):
        pass
    
c = C()  # działa

TypeError: Can't instantiate abstract class B with abstract methods f

In [ ]:
import abc

class A:
    __metaclass__ = abc.ABCMeta
    
    @abc.abstractproperty
    def f(self):
        pass
    

# a = A()  # error

class B(A):
    pass

# b = B()  # error


class C(B):
    @property
    def f(self):
        pass
    
c = C()  # działa

## Weakref

Słabe referencje to obiekty, które nawiązują do innego obiektu, nie nie zwiększają liczby dowiązać (oryginalny obiekt nie ma zwiększonej liczby referencji)

In [27]:
import weakref

class A(object):
    def __init__(self):
        print '{}.__init__'.format(self)
        
    def __del__(self):
        print '{}.__del__'.format(self)
        del self

a = A()

def callback(obj):
    print 'callback: ', obj

aref = weakref.ref(a, callback)
print aref, '; ', aref()  # żeby dostać się do referenta, trzeba wywołać referencję

del a
print aref()  # referent został usunięty, więc referencja pokazuje na None


print 50 * '='
keys_list1 = [A() for i in range(5)]
d1 = weakref.WeakKeyDictionary({a: i for i, a in enumerate(keys_list1)})
print d1, d1.keys()
print len(d1)
keys_list1.pop()
print len(d1)

print 50 * '='
keys_list2 = [A() for i in range(5)]
d2 = weakref.WeakValueDictionary({i: a for i, a in enumerate(keys_list2)})
print d2, d2.values()
print len(d2)
keys_list2.pop()
print len(d2)


print 50 * '='
# dla porównania normalny słownik
keys_list3 = [A() for i in range(5)]
d3 = {a: i for i, a in enumerate(keys_list3)}
print d3, d3.values()
print len(d3)
keys_list3.pop()  # nie wywołuje się __del__
print len(d3)  # dalej jest 5 - bo w słowniku zostały utworzone nowe "twarde" 
               # referencje do obiektów i obiekt nie jest nawet usunięty

    
print 50 * '='
# alternatywnie do słownika można uzyc zbioru
item_list = [A() for i in range(5)]
s = weakref.WeakSet({a for a in item_list})
print s
print len(s)
item_list.pop()
print len(s)

# dodatkowe __del__e w outpucie to usunięte zmienne z poprzedniego uruchomienia

<__main__.A object at 0x7f5bbc704790>.__init__
<weakref at 0x7f5bbc5e6d60; to 'A' at 0x7f5bbc704790> ;  <__main__.A object at 0x7f5bbc704790>
callback:  <weakref at 0x7f5bbc5e6d60; dead>
<__main__.A object at 0x7f5bbc704790>.__del__
None
<__main__.A object at 0x7f5bbc630d90>.__init__
<__main__.A object at 0x7f5bbc62ead0>.__init__
<__main__.A object at 0x7f5bbc62ea90>.__init__
<__main__.A object at 0x7f5bbc62ea10>.__init__
<__main__.A object at 0x7f5bbc62e9d0>.__init__
<WeakKeyDictionary at 140031979509736> [<__main__.A object at 0x7f5bbc630d90>, <__main__.A object at 0x7f5bbc62ead0>, <__main__.A object at 0x7f5bbc62ea90>, <__main__.A object at 0x7f5bbc62e9d0>, <__main__.A object at 0x7f5bbc62ea10>]
5
<__main__.A object at 0x7f5bbc62e9d0>.__del__
4
<__main__.A object at 0x7f5bbc62e9d0>.__init__
<__main__.A object at 0x7f5bbc62eb10>.__init__
<__main__.A object at 0x7f5bbc62eb50>.__init__
<__main__.A object at 0x7f5bbc62eb90>.__init__
<__main__.A object at 0x7f5bbc62ebd0>.__init__
<WeakVa